In [1]:
import wandb
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import f1_score
from wandb.lightgbm import wandb_callback, log_summary

In [2]:
wandb.login()

wandb: Currently logged in as: d-a-pop (midigpt). Use `wandb login --relogin` to force relogin


True

In [3]:
train = pd.read_csv("train.csv")
val = pd.read_csv("val.csv")

In [4]:
train = train.drop("user_id", axis=1)
train = train.drop("book_id", axis=1)
train = train.drop("review_id", axis=1)
train = train.drop("review_text", axis=1)

In [5]:
val = val.drop("user_id", axis=1)
val = val.drop("book_id", axis=1)
val = val.drop("review_id", axis=1)
val = val.drop("review_text", axis=1)

In [51]:
model1 = lgb.LGBMClassifier()

In [65]:
run = wandb.init(project="Goodreads Books Reviews", entity="d-a-pop", job_type="training")

In [66]:
gbm = model1.fit(train.drop("rating", axis=1), train["rating"], callbacks=[wandb_callback()]);
%%time

CPU times: user 10 µs, sys: 4 µs, total: 14 µs
Wall time: 7.87 µs


In [67]:
ypred1_ = model1.predict_proba(val.drop("rating", axis=1))

In [68]:
ypred1 = np.argmax(ypred1_, axis=1)

In [69]:
f1 = f1_score(val['rating'], ypred1, average='macro')

In [70]:
wandb.log({"F1 Score": f1})

In [71]:
# log_summary(gbm)

In [72]:
run.finish()

F1 Score,▁
iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
F1 Score,0.20337
iteration,99


In [34]:
test = pd.read_csv("test.csv")

In [35]:
r = test["review_id"]

In [36]:
print(r)

0         5c4df7e70e9b438c761f07a4620ccb7c
1         8eaeaf13213eeb16ad879a2a2591bbe5
2         dce649b733c153ba5363a0413cac988f
3         8a46df0bb997269d6834f9437a4b0a77
4         d11d3091e22f1cf3cb865598de197599
                        ...               
478028    0e1db3d4b04256f9660f5d276ddf1314
478029    0b7f352e58caf0fd1f961e98ef04e89c
478030    9b19eff33ddb14e9e68fca2e90379e46
478031    8be463fed78f0da63e964706f710332b
478032    62ed1263c7d216986cc419cd4e8a408b
Name: review_id, Length: 478033, dtype: object


In [37]:
type(r)

pandas.core.series.Series

In [38]:
test = test.drop("user_id", axis=1)
test = test.drop("book_id", axis=1)
test = test.drop("review_id", axis=1)
test = test.drop("review_text", axis=1)

In [39]:
p = model1.predict_proba(test)

In [40]:
s = pd.Series(np.argmax(p, axis=1))

In [41]:
s

0         4
1         4
2         4
3         4
4         4
         ..
478028    5
478029    0
478030    4
478031    4
478032    4
Length: 478033, dtype: int64

In [42]:
len(s)

478033

In [43]:
len(test)

478033

In [47]:
submission = pd.concat([r,s],axis=1).rename(columns={0: "rating"})

In [48]:
submission

,review_id,rating
0,5c4df7e70e9b438c761f07a4620ccb7c,4
1,8eaeaf13213eeb16ad879a2a2591bbe5,4
2,dce649b733c153ba5363a0413cac988f,4
3,8a46df0bb997269d6834f9437a4b0a77,4
4,d11d3091e22f1cf3cb865598de197599,4
...,...,...
478028,0e1db3d4b04256f9660f5d276ddf1314,5
478029,0b7f352e58caf0fd1f961e98ef04e89c,0
478030,9b19eff33ddb14e9e68fca2e90379e46,4
478031,8be463fed78f0da63e964706f710332b,4


In [50]:
submission.to_csv("sub2.csv", index=False)